<a href="https://colab.research.google.com/github/alyabdulfatah/classifaction/blob/master/Classification_with_Multivariables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from scipy.io import loadmat

In [29]:
data = loadmat('/content/data.mat')
print(data)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011', '__version__': '1.0', '__globals__': [], 'X': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'y': array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]], dtype=uint8)}


In [30]:
#print(data) 
#print(data['X']) 
#print(data['y'])
#print(data['X'][0]) 
print(data['X'][0][155]) 

print('X Shape = ' ,  data['X'].shape) 
print('Y Shape = ', data['y'].shape)

0.9017626506597092
X Shape =  (5000, 400)
Y Shape =  (5000, 1)


In [0]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [0]:
def cost(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg


In [0]:
def gradient_with_loop(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learningRate / len(X)) * theta[:,i])
    
    return grad

In [0]:
def gradient(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()

In [0]:
from scipy.optimize import minimize

In [0]:
def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0] #5000
    params = X.shape[1] #400
    
    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    print('all_theta shape ' , all_theta.shape)
    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    print('X shape ' , X.shape)
    
    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        
        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
    
    return all_theta


In [37]:
rows = data['X'].shape[0]    #5000 in data
params = data['X'].shape[1]  #400

print('rows = ' ,rows)
print('params = ' , params)


rows =  5000
params =  400


In [38]:
all_theta = np.zeros((10, params + 1))

print('all_theta \n' , all_theta)
print('all_theta shape \n' , all_theta.shape)

all_theta 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
all_theta shape 
 (10, 401)


In [39]:
X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)
print('X Shape = ' ,  X.shape) 

X Shape =  (5000, 401)


In [0]:
theta = np.zeros(params + 1)

In [41]:
y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
print(y_0.shape)

(5000,)


In [42]:
y_0 = np.reshape(y_0, (rows, 1))
print(y_0.shape)

(5000, 1)


In [43]:
print()
print('X.shape = ',X.shape)
print()
print('y.shape = ',y_0.shape)
print()
print('theta.shape = ',theta.shape)
print()
print('all_theta.shape = ',all_theta.shape)

print()
print('data array = ' , np.unique(data['y']))
print()


X.shape =  (5000, 401)

y.shape =  (5000, 1)

theta.shape =  (401,)

all_theta.shape =  (10, 401)

data array =  [ 1  2  3  4  5  6  7  8  9 10]



In [54]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)

print('Theta shape =   ' , all_theta.shape)


all_theta shape  (100, 401)
X shape  (5000, 401)
Theta shape =    (100, 401)


In [55]:
#print('Theta = ')
#print(all_theta)


Theta = 
[[-3.70247931e-05  0.00000000e+00  0.00000000e+00 ... -2.24803602e-10
   2.31962907e-11  0.00000000e+00]
 [-8.96250716e-05  0.00000000e+00  0.00000000e+00 ...  7.26120810e-09
  -6.19965284e-10  0.00000000e+00]
 [-8.39553387e-05  0.00000000e+00  0.00000000e+00 ... -7.61695633e-10
   4.64917656e-11  0.00000000e+00]
 ...
 [-2.93489055e-04  0.00000000e+00  0.00000000e+00 ... -7.79980562e-10
  -3.04839302e-11  0.00000000e+00]
 [-2.93489055e-04  0.00000000e+00  0.00000000e+00 ... -7.79980562e-10
  -3.04839302e-11  0.00000000e+00]
 [-2.93489055e-04  0.00000000e+00  0.00000000e+00 ... -7.79980562e-10
  -3.04839302e-11  0.00000000e+00]]


In [0]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)
    
    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)
    
    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1
    
    return h_argmax

In [0]:
y_pred = predict_all(data['X'], all_theta)

In [58]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 74.6%
